In [1]:
import pandas
import geopandas
import matplotlib.pyplot as plt
from fuzzywuzzy import process

In [25]:
df = pandas.read_csv("./GA_precincts_with_absentee.csv")

In [26]:
gdf = geopandas.read_file("./shapefiles/GA_precincts16.shp")

In [27]:
def search_for_matches(county):
    choices = gdf[gdf["CTYNAME"] == county]["PRECINCT_N"]
    return df[df["COUNTY"] == county]["PRECINCT"].apply(lambda name: process.extractOne(name, choices))

In [28]:
counties = df["COUNTY"].unique()

In [29]:
df.columns

Index(['Unnamed: 0', 'COUNTY', 'PRECINCT', 'PRES16D', 'PRES16D_AB',
       'PRES16D_AD', 'PRES16D_ED', 'PRES16D_PR', 'PRES16L', 'PRES16L_AB',
       'PRES16L_AD', 'PRES16L_ED', 'PRES16L_PR', 'PRES16R', 'PRES16R_AB',
       'PRES16R_AD', 'PRES16R_ED', 'PRES16R_PR', 'REG_VOTE'],
      dtype='object')

In [31]:
(gdf.groupby("CTYNAME").size() == 1).sum()

12

In [32]:
(df.groupby("COUNTY").size() == 1).sum()

12

In [33]:
df["COUNTY"].nunique() - gdf["CTYNAME"].nunique()

0